In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import HTML
from io import BytesIO
import base64
from matplotlib.colors import ListedColormap
from scipy.interpolate import griddata
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.patches import Circle, PathPatch
import matplotlib.patheffects as PathEffects

# 맵 이미지를 Base64 문자열로 인코딩하는 함수 - 수정된 버전
def plot_to_base64(x, y, values):
    # 색상을 결정하기 위한 조건 설정
    colors = np.where(values < 0.3, 'red', np.where(values > 0.3, 'blue', 'gray'))
    
    fig, ax = plt.subplots(figsize=(2, 2), dpi=100)
    # 사각형 마커와 조건에 따른 색상 사용
    scatter = ax.scatter(x, y, s=100, c=colors, marker='s', edgecolor='black')
    ax.axis('off')
    
    buf = BytesIO()
    fig.savefig(buf, format='png', bbox_inches='tight', pad_inches=0)
    plt.close(fig)
    return base64.b64encode(buf.getvalue()).decode()

# HTML 테이블 생성 및 맵 이미지 삽입 함수 - "Map" 문자열 변경
def create_html_table_with_maps(df):
    # CSS 스타일 정의 (테이블 및 이미지 크기 조정 포함)
    style = """
    <style>
        table {
            width: 40%; /* 테이블 너비 조정 */
            border-collapse: collapse;
            margin: 10px auto; /* 중앙 정렬 */
            font-family: Arial, sans-serif;
        }
        th, td {
            padding: 4px; /* 패딩 조정 */
            border: 1px solid #ccc;
            text-align: center;
            font-size: 14px; /* 글자 크기 조정 */
        }
        th {
            background-color: #f0f0f0;
        }
        .value {
            text-align: center;
            font-weight: bold;
        }
        img {
            width: 100%; /* 이미지 너비 셀에 꽉 차게 조정 */
            height: auto; /* 이미지 높이 자동 조정 */
        }
        .map-cell {
            width: 50px; /* 맵 셀 너비 조정 */
            height: 50px; /* 맵 셀 높이 조정, 정사각형 유지 */
        }
    </style>
    """

    html_str = style + '<table>'
    cat3_values = sorted(df['cat3'].unique())
    
    # 헤더 생성
    html_str += '<tr><td class="value">Value</td>' + ''.join([f'<td>{cat3}</td>' for cat3 in cat3_values]) + '</tr>'
    
    # 각 val 컬럼에 대해 행 추가
    for i, val in enumerate([f'val{i}' for i in range(1, 6)], start=1):
        html_str += f'<tr><td class="value">{val}</td>'
        for cat3 in cat3_values:
            mean_val = df[df['cat3'] == cat3][val].mean()
            html_str += f'<td>{mean_val:.2f}</td>'
        html_str += '</tr>'
        
        # 맵 이미지 삽입, "Map" 문자열을 "valX_map"으로 변경
        html_str += f'<tr><td class="value">{val}_map</td>'
        for cat3 in cat3_values:
            subset = df[df['cat3'] == cat3]
            img_data = plot_to_base64(subset['x'], subset['y'], subset[val])
            html_str += f'<td><img src="data:image/png;base64,{img_data}"></td>'
        html_str += '</tr>'
    
    html_str += '</table>'
    return html_str

    stats_html = summary_stats.to_html()
    boxplot_html = f'<img src="data:image/png;base64,{boxplot_img_data}" style="max-width:100%;">'
    return stats_html + '<br>' + boxplot_html

# 데이터 프레임 생성
np.random.seed(0)
data = {
    'cat1': np.random.choice(['ProductA', 'ProductB', 'ProductC', 'ProductD', 'ProductE', 'ProductF', 'ProductG'], 3000),
    'cat2': np.random.choice(['Setting1', 'Setting2', 'Setting3', 'Setting4', 'Setting5', 'Setting6', 'Setting7', 'Setting8'], 3000),
    'cat3': np.random.randint(1, 11, 3000),
    'x': np.random.randint(1, 11, 3000),
    'y': np.random.randint(1, 11, 3000),
    'datetime': pd.date_range(start='2021-01-01', periods=3000, freq='D')
}
df = pd.DataFrame(data)
vals = {f'val{val_num}': np.random.random(3000) for val_num in range(1, 6)}
df = pd.concat([df, pd.DataFrame(vals)], axis=1)

# cat1이 'ProductA'인 행만 필터링
filtered_df = df[df['cat1'] == 'ProductA']

# 1. 데이터프레임에서 HTML 테이블과 맵 이미지 생성
html_table_with_maps = create_html_table_with_maps(filtered_df)

HTML(html_table_with_maps)

# 박스플롯 생성 및 인코딩
def create_boxplot_base64(df, val_column):
    plt.figure(figsize=(10, 3))  # 박스플롯 크기 조절
    sns.boxplot(x='cat3', y=val_column, data=df)
    plt.title(f'{val_column} by cat3')
    buf = BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight')
    plt.close()
    return base64.b64encode(buf.getvalue()).decode()

# 통계치 계산 및 HTML 포맷팅
def calculate_and_format_stats_html(df, val_columns):
    html_str = "<style>table {border-collapse: collapse; width: 80%;} th, td {border: 1px solid #ddd; text-align: left; padding: 8px;} th {background-color: #f2f2f2;}</style>"
    for val_column in val_columns:
        stats_df = df.groupby('cat3')[val_column].describe().reset_index()
        stats_df['<0.5 count'] = df[df[val_column] < 0.5].groupby('cat3')[val_column].count().values
        stats_html = stats_df.to_html(index=False)
        html_str += f"<h3>{val_column} Statistics</h3>" + stats_html
    return html_str

# 전체 HTML 컨텐츠 생성 및 파일 저장
def create_full_html_content(df):
    val_columns = [f'val{i}' for i in range(1, 6)]
    full_html_content = ""
    for val_column in val_columns:
        boxplot_img_data = create_boxplot_base64(df, val_column)
        full_html_content += f'<img src="data:image/png;base64,{boxplot_img_data}" style="width:60%; display: block; margin-left: auto; margin-right: auto;">'
        stats_html = calculate_and_format_stats_html(df, [val_column])
        full_html_content += stats_html
    return full_html_content

html_content = create_full_html_content(filtered_df)

full_html_content = "<style>table {width: 100%; border-collapse: collapse;} th, td {border: 1px solid #ddd; padding: 8px; text-align: center;} th {background-color: #f2f2f2;}</style>"
full_html_content += html_table_with_maps
full_html_content += html_content

HTML(full_html_content)
